# CUP

In [39]:
# cup is measured in centimeters becasue cm^3 = 1 ml
material = ["ceramic", "glass", "stainless steel", "Paper", "styrofoam", "plastic"]
avg_cost = [0.0005381957526 , 0.02690978763, 0.1614587258, 0.0006458349031, 0.0004736122623, 0.006673627332] #per sqr foot convert to per sqr centimeter
# enviormental cost of producing each of these materials

# make a handle of the cup with an empty volume using the equations of a half torus
ration_hieght_to_handle_hieght = 73/95
radius_to_length_ratio = 14/45

daily_water_intake = 3700
min_thickness = 0.5
max_thickness = 0.7
hight_to_width_ratio = 82/95 # height to width raio of a mug must stay constant when creating this mug

using JuMP, Ipopt
# m = Model(Cbc.Optimizer)
m = Model(Ipopt.Optimizer)

@variable(m, r >= 0)
@variable(m, h >= 0)
@variable(m, r_handle >= 0)
@variable(m, R >= 0)
@variable(m, surf)
#@variable(m, bottom_thickness >= 1)
#@variable(m, min_thickness >= padding >= max_thickness)
@constraint(m, hight_to_width_ratio * h >= r) # the cup must maintian the hight to radius ratio 
@constraint(m, hight_to_width_ratio * h <= r)
@constraint(m, (ration_hieght_to_handle_hieght * h)/2 >= R) # the cup must maintian the hight to radius ratio 
@constraint(m, (ration_hieght_to_handle_hieght * h)/2 <= R)
@constraint(m, radius_to_length_ratio * R >= r_handle) # the cup must maintian the hight to radius ratio 
@constraint(m, radius_to_length_ratio * R <= r_handle)
@NLconstraint(m, π^2 *R * r_handle^2  + π*r^2*h == daily_water_intake/3) # volume of the cup must allow me to drink it 3 times to make my daily intake
#@NLconstraint(m, π^2 *R * r_handle^2  + π*r^2*h <= daily_water_intake/3)

@NLconstraint(m, surf >= 2 * π * r^2 + 2 * π * r * h +  2* π^2 *R * r_handle ) # min-max problem

#@NLexpression(m, surface_area, 2 * π * (r+padding)^2 + 2 * π * (r + padding) * (h + bottom_thickness)) # surface area of a cylinder of the cup size to calculate the cost of material
@NLexpression(m, cost[i in 1:6], surf*avg_cost[i])
@NLobjective(m, Min, sum(cost[i] for i in 1:6))

optimize!(m)


This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:       17
Number of nonzeros in Lagrangian Hessian.............:       12

Total number of variables............................:        5
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        7
        inequality constraints with only lower bounds:        4
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        3

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [43]:
println("radius of the optimal cup is: ", value.(r))
println("hieght of the optimal cup is: ", value.(h))
println("maximum surface area of the cup is: " ,value.(surf))
println("the minimum cost of the each material cup: ", value.(cost))
println("the radius of the hollow handle is: ", value.(r_handle))
println("the overall radius of the cupe handle: ", value.(R))
println("the overall volume of the cup is: ", π^2 *value.(R) * value.(r_handle)^2  + π*value.(r)^2*value.(h))
println("price of all cups: ", objective_value(m))

radius of the optimal cup is: 6.918747814735494
hieght of the optimal cup is: 8.015622477970313
maximum surface area of the cup is: 707.468530220912
the minimum cost of the each material cup: [0.3807565580630596, 19.03782790315298, 114.22696743306726, 0.4569078696615222, 0.33506577110398206, 4.721381319812147]
the radius of the hollow handle is: 0.9581246858916144
the overall radius of the cupe handle: 3.0796865214632985
the overall volume of the cup is: 1233.3333333333335
price of all cups: 139.15890685486096


In [64]:
import Pkg;
Pkg.add("Plots")
using Plots

x = [i for i in 1:100]
y = [i for i in 1:100]
z = [(x[i]*x[i])/value.(r) + ((y[i]-value.(h))*(y[i]-value.(h)))/.3 for i in 1:100]
plot(x,y,z)

  Resolving package versions...


LoadError: Unable to automatically install 'libass' from '/Users/jasbakshi/.julia/packages/libass_jll/uUbBl/Artifacts.toml'

In [54]:
using PyPlot
n = 100
u = range(0,stop=2*π,length=n);
v = range(0,stop=π,length=n);

x = cos.(u) .* sin.(v);
y = sin.(u) .* sin.(v);
z = ones(n) .* cos.(v);

# The rstride and cstride arguments default to 10
plot( diag(x), diag(y), diag(z), color="yellow", linewidth=3 )

LoadError: UndefVarError: diag not defined